In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade tensorflow
!pip install keras

In [ ]:
import numpy as np
import math
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import json
import re

In [ ]:
#mediapipeのデータを正規化
def readMedipipeToRow(json_file):
  data = json.load(json_file)
  #ランドマークの座標を一列の配列に変換
  coordinates = []
  max = -1
  x1 = 0.0
  y1 = 0.0
  z1 = 0.0
  for landmark in data[0]['landmarks']:
    print(landmark)
    if landmark['name'] == 'WRIST':
      x1 = landmark['x']
      y1 = landmark['y']
      z1 = landmark['z']
    else:
      t = pow(landmark['x']-x1, 2) + pow(landmark['y']-y1, 2) + pow(landmark['z']-z1, 2)
      if t > max: max = t
      print(max)
      new_landmark = [landmark['x'] - x1, landmark['y'] - y1, landmark['z'] - z1]  # (1, 3) の形にする
      # axis=0 で新しい行を追加
      print(new_landmark)
      coordinates.append(new_landmark)
  return coordinates/math.sqrt(max)

In [ ]:
#正規化したデータを配列に変換
directory = '/content/drive/Shareddrives/JPHackathon-241027/単語手話/annotated_points'
x = []
y = []
# 指定したディレクトリ内の全てのサブディレクトリを取得
subdirs = sorted(
    [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))],
    key=lambda x: int(x) if x.isdigit() else x
)

# 各サブディレクトリを一つずつ処理
for subdir in subdirs:
    subdir_path = os.path.join(directory, subdir)
    print(f"Processing directory: {subdir_path}")

    # サブディレクトリ内のファイルを取得
    for file in os.listdir(subdir_path):
        # JSONファイルを探す
        if file.endswith('.json'):
            file_path = os.path.join(subdir_path, file)
            try:
                # JSONファイルを読み込み、内容を出力
                with open(file_path, 'r', encoding='utf-8') as json_file:
                  data = json.load(json_file)
                  #ランドマークの座標を一列の配列に変換
                  coordinates = []
                  max = -1
                  x1 = 0.0
                  y1 = 0.0
                  z1 = 0.0
                  for landmark in data[0]['landmarks']:
                    if landmark['name'] == 'WRIST':
                      x1 = landmark['x']
                      y1 = landmark['y']
                      z1 = landmark['z']
                    else:
                      t = pow(landmark['x']-x1, 2) + pow(landmark['y']-y1, 2) + pow(landmark['z']-z1, 2)
                      if t > max: max = t
                      new_landmark = [landmark['x'] - x1, landmark['y'] - y1, landmark['z'] - z1]  # (1, 3) の形にする
                      # axis=0 で新しい行を追加
                      coordinates = np.append(coordinates, new_landmark)
                  coordinates = np.array(coordinates) / math.sqrt(max)
                  x.append(coordinates)
                  #x.append(readMedipipeToRow(json_file))
                  match = re.match(r"(\d+)-[a-z]+", subdir)
                  if match:
                    number = match.group(1)
                    number = int(number)
                  y.append(number)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

# NumPy配列に変換
x = np.array(x)
y = np.array(y)

print(x)
print(y)

# データの分割
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


Processing directory: /content/drive/Shareddrives/JPHackathon-241027/単語手話/annotated_points/00-a
Processing directory: /content/drive/Shareddrives/JPHackathon-241027/単語手話/annotated_points/01-i
Processing directory: /content/drive/Shareddrives/JPHackathon-241027/単語手話/annotated_points/02-u
Processing directory: /content/drive/Shareddrives/JPHackathon-241027/単語手話/annotated_points/03-e
Processing directory: /content/drive/Shareddrives/JPHackathon-241027/単語手話/annotated_points/04-o
Processing directory: /content/drive/Shareddrives/JPHackathon-241027/単語手話/annotated_points/05-ka
Processing directory: /content/drive/Shareddrives/JPHackathon-241027/単語手話/annotated_points/06-ki
Processing directory: /content/drive/Shareddrives/JPHackathon-241027/単語手話/annotated_points/07-ku
Processing directory: /content/drive/Shareddrives/JPHackathon-241027/単語手話/annotated_points/08-ke
Processing directory: /content/drive/Shareddrives/JPHackathon-241027/単語手話/annotated_points/09-ko
Processing directory: /content/driv

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler

# データの標準化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# モデルの構築
model = Sequential([
    Dense(128, activation='relu', input_shape=(60,)),
    Dense(64, activation='relu'),
    Dense(len(set(y)), activation='softmax')
])

# モデルのコンパイル
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# モデルの訓練
model.fit(X_train, y_train, epochs=100, batch_size=32)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2792 - loss: 2.4423    
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7054 - loss: 1.3338 
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8314 - loss: 0.8957  
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9023 - loss: 0.6360 
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9276 - loss: 0.4745 
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9550 - loss: 0.3293 
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9609 - loss: 0.2512 
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9746 - loss: 0.1931 
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9768 - loss: 0.1587 
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9748 - loss: 0.1414 
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9776 - loss: 0.1070 
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s

In [ ]:
# モデルの評価
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9717 - loss: 0.3721  
Test accuracy: 0.9658119678497314


In [ ]:
# 新しいデータに対する予測
predictions = model.predict(X_test)
print(predictions[0][1])
predicted_classes = np.argmax(predictions, axis=1)
print(f'Predicted classes: {predicted_classes}')
print(y_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
7.644671e-10
Predicted classes: [14 15  3 15 15 11  4 10  5 13 15  3 15  7 15  8  8  4 15  7 13  8  0 15
 15  2  2 15 15  1  6 15  3  9  3 12 15 14 15 15  5  8  6  0 15 13  0 15
 15 15  4  6  3  7  7  8 15 15 15 15  3  8 12  2  3 11  9 15 13  1  8 12
  6  3  7 15  2  3  7 15 15  0  3 13 15  3 15 15 10 10  5 15  6  7 15 15
 11  0 13 15 13 15  9  3 15 15  9 15  5  7  6  4 11 15  6 15 13]
[14 15  3 15 15 11  4 10  5 13 15  3 15  7 15  8  8  4 15  7 13  8  0 15
 15  2  2 15 15  1  6 15  3  9  3 12 15 14 15 15  9  8  6  0 15 13  0 15
 15 15  4  6  3  7  7  8 15 15 15 15  3  8 12  1  3 11  9 15 13  1  5 12
  6  3 14 15  2  3  7 15 15  0  3 13 15  3 15 15 10 10  5 15  6  7 15 15
 11  0 13 15 13 15  9  3 15 15  9 15  5  7  6  4 11 15  6 15 13]


Tensorflow liteを保存して実行する

In [ ]:
folder = '/content/drive/Shareddrives/JPHackathon-241027/単語手話/classify hands/'
model.save(folder + 'a-ta-ikami-mikubo-model-edit.keras')

# Keras モデルを TensorFlow Lite モデルに変換
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # 直接モデルを指定
tflite_model = converter.convert()

open(folder + "a-ta-ikami-mikubo-model-edit.tflite", "wb").write(tflite_model)

Saved artifact at '/tmp/tmphm7jvmia'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 60), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 16), dtype=tf.float32, name=None)
Captures:
  135943787457952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135943787461648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135943787070912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135943787073200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135943787069680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135943787074784: TensorSpec(shape=(), dtype=tf.resource, name=None)


70500

In [ ]:
# TFLiteモデルの読み込み
interpreter = tf.lite.Interpreter(model_path= folder + "a-ta-ikami-mikubo-model.tflite")
# メモリ確保。これはモデル読み込み直後に必須
interpreter.allocate_tensors()
# 学習モデルの入力層・出力層のプロパティをGet.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
print(input_details[0]['shape'])

[ 1 60]


In [ ]:
print(input_details[0]['index'])

0


In [ ]:
# X_testからデータを取得し、型をfloat32に変換
input_data = X_test[0].astype(np.float32).reshape(input_details[0]['shape'])  # リシェイプ
interpreter.set_tensor(input_details[0]['index'], input_data)

# 推論の実行
interpreter.invoke()

# 出力データの取得
output_data = interpreter.get_tensor(output_details[0]['index'])

# 結果の表示
print(f"Output for test sample {0}:", output_data)

Output for test sample 0: [[4.2194247e-07 1.6062433e-08 6.3411608e-05 3.3206138e-08 1.0473051e-09
  1.4449451e-05 2.6188829e-06 6.7762386e-11 2.2622806e-09 3.5720618e-06
  8.4131226e-09 6.0215520e-05 3.2248539e-05 5.1620140e-12 9.9982303e-01
  1.4711342e-09]]


In [ ]:
print(y_test[0])

14
